# Job Application Tailor

* **Goal**:

	* Maximize chances of getting an interview for a given job posting

* **Process**:

	* Learn about the job requirements
	* Cross that with your skill set and experiences
	* Reshape resume to highlight relevant areas
	* Rewrite resume with appropriate language
	* Set talking points for your initial interview

In [1]:
import warnings

warnings.filterwarnings("ignore")

In [2]:
from crewai import Agent, Task, Crew

In [3]:
import os
from dotenv import load_dotenv

dotenv_path = os.path.abspath(os.path.join(os.getcwd(), "..", ".env"))

load_dotenv(dotenv_path)

openai_api_key = os.getenv("OPENAI_API_KEY")
openai_model_name = os.getenv("OPENAI_MODEL_NAME")

## Tools

In [ ]:
from crewai_tools import FileReadTool, ScrapeWebsiteTool, MDXSearchTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
read_resume = FileReadTool(file_path="./resume.md")
semantic_search_resume = MDXSearchTool(mdx="./resume.md")

/Users/marconardoneguerra/development/CrewAI/.venv/lib/python3.13/site-packages/alembic/config.py:592: DeprecationWarning: No path_separator found in configuration; falling back to legacy splitting on spaces, commas, and colons for prepend_sys_path.  Consider adding path_separator=os to Alembic config.
  util.warn_deprecated(
Inserting batches in chromadb:   0%|          | 0/1 [00:00<?, ?it/s]/Users/marconardoneguerra/development/CrewAI/.venv/lib/python3.13/site-packages/chromadb/types.py:144: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  return self.model_fields  # pydantic 2.x
Inserting batches in chromadb: 100%|██████████| 1/1 [00:01<00:00,  1.55s/it]


## Agents

In [ ]:
researcher = Agent(
    role="Tech Job Researcher",
    goal="Make sure to do amazing analysis on job posting to help job applicants",
    tools=[scrape_tool, search_tool],
    verbose=True,
    backstory=(
        "As a Job Research, your prowess in navigating "
        "and extracting critical information "
        "from job postings is unmatched."
        "Your skills help pinpoint the necessary "
        " qualifications and skills sought by employers, "
        "forming the foundation for effective application tailoring."
    ),
)

In [ ]:
profiler = Agent(
    role="Personal Profiler for Engineers",
    goal="Do incredible research on job applicants"
    "to help them stand out in the job market",
    tools=[scrape_tool, search_tool, read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Equipped with analytical prowess, "
        "you dissect and synthesize information "
        "from diverse sources to craft comprehensive "
        "personal and professional profiles, "
        "laying the groundwork for personalized resume enhancements."
    ),
)

In [ ]:
resume_strategist = Agent(
    role="Resume Strategist for Engineers",
    goal="Find all the best ways to make a resume stand out in the job market.",
    tools=[scrape_tool, search_tool, read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "With a strategic mind and an eye for detail, "
        "you excel at refining resumes to highlight "
        "the most relevant skills and experiences, "
        "ensuring they resonate perfectly with "
        "the job's requirements."
    ),
)

In [20]:
editor = Agent(
    role="Resume Editor",
    goal="Refine the resume created by the resume strategist to maintain neatness and proper md formatting",
    tools=[read_resume],
    verbose=True,
    backstory=(
        """
		You excel at reviewing resumes for professionals to get them the job they want.
		You take a resume draft and refine it for viewing for recruiters and/or HR reviewing job applicants.
		"""
    ),
)

In [ ]:
interview_preparer = Agent(
    role="Engineering Interview Preparer",
    goal="Create interview questions and talking points "
    "based on the resume and job requirements",
    tools=[scrape_tool, search_tool, read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Your role is crucial in anticipating "
        "the dynamics of interviews. "
        "With your ability to formulate key questions "
        "and talking points, "
        "you prepare candidates for success, "
        "ensuring they can confidently "
        "address all aspects of the job they are applying for."
    ),
)

## Tasks

In [ ]:
research_task = Task(
    description=""" Analyze the job posting URL provided ({job_posting_url}) to extract key skills,
					  experiences, and qualifications required. Use the tools to gather content and
					  identify and categorize the requirements.""",
    expected_output="""A structured list of job requirements, including necessary skills, qualifications, and experiences.""",
    agent=researcher,
    async_execution=True,
)

In [ ]:
profile_task = Task(
    description=(
        "Compile a detailed personal and professional profile "
        "using the GitHub ({github_url}) URLs, "
        "and personal write-up ({personal_writeup}). "
        "Utilize tools to extract and synthesize information "
        "from these sources."
    ),
    expected_output=(
        "A comprehensive profile document that "
        "includes skills, project experiences, "
        "contributions, interests, and communication style."
    ),
    agent=profiler,
    async_execution=True,
)

In [ ]:
resume_strategy_task = Task(
    description=(
        "Using the profile and job requirements "
        "obtained from previous tasks, "
        "tailor the resume to highlight the most relevant areas. "
        "Employ tools to adjust and enhance the resume content. "
        "Make sure this is the best resume ever "
        "but don't make up any information. "
        "Update every section, including the initial summary, "
        "work experience, skills, and education. "
        "All to better reflect the candidate's "
        "abilities and how it matches the job posting."
    ),
    expected_output=(
        "An updated resume that effectively "
        "highlights the candidate's qualifications "
        "and experiences relevant to the job."
    ),
    output_file="tailored_resume.md",
    context=[
        research_task,
        profile_task,
    ],  # require previous two tasks to be executed before executing this task
    agent=resume_strategist,
)

In [ ]:
# TO-DO: resume refinement task

resume_refinement_task = Task(
    description=(
        """ 
		
		"""
    )
)

In [ ]:
interview_preparation_task = Task(
    description=(
        "Create a set of potential interview questions "
        "and talking points based "
        "on the tailored resume and job requirements. "
        "Utilize tools to generate relevant questions "
        "and discussion points. "
        "Make sure to use these question and talking points "
        "to help the candidate "
        "highlight the main points of the resume "
        "and how it matches the job posting."
    ),
    expected_output=(
        "A document containing key questions and "
        "talking points that the candidate "
        "should prepare for the initial interview."
    ),
    output_file="interview_materials.md",
    context=[research_task, profile_task, resume_strategy_task],
    agent=interview_preparer,
)

## Crew

In [ ]:
job_application_crew = Crew(
    agents=[researcher, profiler, resume_strategist, interview_preparer],
    tasks=[
        research_task,
        profile_task,
        resume_strategy_task,
        interview_preparation_task,
    ],
    verbose=True,
)

In [ ]:
job_application_inputs = {
    "job_posting_url": "https://jobs.lever.co/AIFund/6c82e23e-d954-4dd8-a734-c0c2c5ee00f1?lever-origin=applied&lever-source%5B%5D=AI+Fund",
    "github_url": "https://github.com/majinmarco",
    "personal_writeup": """Results‑driven AI Engineer and Data Scientist with end‑to‑end experience across ML, LLM applications, data engineering, and analytics. Blend of technical depth (Python, Databricks, Spark, Vertex AI, SQL) and client‑facing consulting (requirements, BRDs, Agile PM). Delivered demand sensing pipelines, RAG chatbots, proposal generators, and recommendation systems that improved forecast accuracy, sped up operations, and drove revenue growth. Recognized as a **Top Performer (2023–2024)** for innovation and impact.""",
}

# Crew kickoff

In [19]:
job_application_crew.kickoff(inputs=job_application_inputs)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 508f40e3-8dbd-41e6-9e75-a4d95d77f1e0                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Personal Profiler for Engineers                                                                         │
│                                                                                                                 │
│  Task: Compile a detailed personal and professional profile using the GitHub (https://github.com/majinmarco)    │
│  URLs, and personal write-up (Results‑driven AI Engineer and Data Scientist with end‑to‑end experience across   │
│  ML, LLM applications, data engineering, and analytics. Blend of technical depth (Python, Databricks, Spark,    │
│  Vertex AI, SQL) and client‑facing consulting (requirements, BRDs, Agile PM). Delivered demand sensing          │
│  pipelines, RAG chatbots, proposal generators, and recommendation systems that improved forecast accuracy,      │
│  sped up operations, and drove revenue growth. Recognized as a **Top Performer (2023–2024)** for innovation     │
│  and impact.). Utilize tools to extract and synthesize information from these sources.                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Tech Job Researcher                                                                                     │
│                                                                                                                 │
│  Task:  Analyze the job posting URL provided                                                                    │
│  (https://jobs.lever.co/AIFund/6c82e23e-d954-4dd8-a734-c0c2c5ee00f1?lever-origin=applied&lever-source%5B%5D=AI  │
│  +Fund) to extract key skills,                                                                                  │
│                                            experiences, and qualifications required. Use the tools to gather    │
│  content and                                                                                                    │
│                                            identify and categorize the requirements.                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

...

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Engineering Interview Preparer                                                                          │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  A. Interview questions by theme (with guidance on how to answer using the resume content)                      │
│  1) Role Fit and Value Proposition                                                                              │
│  - Tell us about your interest in venture capital/private equity or corporate development, and how your         │
│  background in AI engineering and data science translates to investment decisions in AI/tech companies.         │
│    Talking points to highlight from resume:                                                                     │
│    - End-to-end ML pipelines, data engineering, and RAG chatbots, showing strong tech intuition and ability to  │
│  assess technology risk.                                                                                        │
│    - Experience translating analytics into business outcomes (forecast accuracy, revenue impact), aligning      │
│  with investment theses and value creation.                                                                     │
│    - Top Performer recognition (2023–2024) for innovation and impact, illustrating a track record of            │
│  delivering measurable results.                                                                                 │
│  - Why this firm (or the stated role) and how do you see your skill set contributing to our deal flow and       │
│  portfolio value?                                                                                               │
│    Talking points:                                                                                              │
│    - BRD creation, requirements elicitation, Agile delivery, and client-facing analytics experience show you    │
│  can source, diligence, and support investments with structured analyses.                                       │
│    - Market sizing, competitive landscape analysis, and data-driven decision support align with building        │
│  strong investment theses.                                                                                      │
│    - Interest in AI/technology trends and willingness to work in fast-paced, collaborative environments map to  │
│  the “high-growth tech, AI focus” preference.                                                                   │
│                                                                                                                 │
│  2) Technical/Analytical Skills and Modeling                                                                    │
│  - Describe a project where you built an end-to-end ML solution and explain how you measured impact (e.g.,      │
│  forecast accuracy, operational efficiency, ROI).                                                               │
│    Talking points:                                                                                              │
│    - Provide a concise narrative of data ingestion, feature engineering, model training, deployment, and        │
│  monitoring in cloud environments (Databricks, Vertex AI).                                                      │
│    - Quantify impact if possible (e.g., improved forecast accuracy, time-to-insight reductions, revenue impact  │
│  through decision support).                                                                                     │
│    - Link to investment analytics by framing model outc

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 5fa7493a-87f3-4999-9fd8-becc0649b29f                                                                     │
│  Agent: Engineering Interview Preparer                                                                          │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 508f40e3-8dbd-41e6-9e75-a4d95d77f1e0                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: A. Interview questions by theme (with guidance on how to answer using the resume content)        │
│  1) Role Fit and Value Proposition                                                                              │
│  - Tell us about your interest in venture capital/private equity or corporate development, and how your         │
│  background in AI engineering and data science translates to investment decisions in AI/tech companies.         │
│    Talking points to highlight from resume:                                                                     │
│    - End-to-end ML pipelines, data engineering, and RAG chatbots, showing strong tech intuition and ability to  │
│  assess technology risk.                                                                                        │
│    - Experience translating analytics into business outcomes (forecast accuracy, revenue impact), aligning      │
│  with investment theses and value creation.                                                                     │
│    - Top Performer recognition (2023–2024) for innovation and impact, illustrating a track record of            │
│  delivering measurable results.                                                                                 │
│  - Why this firm (or the stated role) and how do you see your skill set contributing to our deal flow and       │
│  portfolio value?                                                                                               │
│    Talking points:                                                                                              │
│    - BRD creation, requirements elicitation, Agile delivery, and client-facing analytics experience show you    │
│  can source, diligence, and support investments with structured analyses.                                       │
│    - Market sizing, competitive landscape analysis, and data-driven decision support align with building        │
│  strong investment theses.                                                                                      │
│    - Interest in AI/technology trends and willingness to work in fast-paced, collaborative environments map to  │
│  the “high-growth tech, AI focus” preference.                                                                   │
│                                                                                                                 │
│  2) Technical/Analytical Skills and Modeling                                                                    │
│  - Describe a project where you built an end-to-end ML solution and explain how you measured impact (e.g.,      │
│  forecast accuracy, operational efficiency, ROI).                                                               │
│    Talking points:                                                                                              │
│    - Provide a concise narrative of data ingestion, feature engineering, model training, deployment, and        │
│  monitoring in cloud environments (Databricks, Vertex AI).                                                      │
│    - Quantify impact if possible (e.g., improved forecast accuracy, time-to-insight reductions, revenue impact  │
│  through decision support).                           

CrewOutput(raw='A. Interview questions by theme (with guidance on how to answer using the resume content)\n1) Role Fit and Value Proposition\n- Tell us about your interest in venture capital/private equity or corporate development, and how your background in AI engineering and data science translates to investment decisions in AI/tech companies.\n  Talking points to highlight from resume:\n  - End-to-end ML pipelines, data engineering, and RAG chatbots, showing strong tech intuition and ability to assess technology risk.\n  - Experience translating analytics into business outcomes (forecast accuracy, revenue impact), aligning with investment theses and value creation.\n  - Top Performer recognition (2023–2024) for innovation and impact, illustrating a track record of delivering measurable results.\n- Why this firm (or the stated role) and how do you see your skill set contributing to our deal flow and portfolio value?\n  Talking points:\n  - BRD creation, requirements elicitation, Agil